In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,linkedlist,update_topology,spanning_tree

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [2]:
# Query for ACLineSegment, Transformerend, Switches, Breakers, Reclosers, Fuses, Sectionalisers,SynchronousMachine,TopologicalNode
topologyqueries.getallqueries(gapps,model_mrid)
Line_query=topologyqueries.Line_query
XfmrDict=topologyqueries.XfmrDict
XfmrKeys=topologyqueries.XfmrKeys
SwitchDict=topologyqueries.SwitchDict
SwitchKeys=topologyqueries.SwitchKeys
DG_query=topologyqueries.DG_query
Cap_query=topologyqueries.Cap_query
Node_query=topologyqueries.Node_query

AttributeError: module 'topologyqueries' has no attribute 'Cap_query'

In [ ]:
# Build Linknet Lists from ACLineSegment,Transformers,DGs and Nodes
[ConnNodeDict,TerminalsDict,TermList,NodeList]=linkedlist.build_linked_list(Line_query,XfmrDict,XfmrKeys,DG_query,Node_query)

# Stash a copy of base dictionary
BaseConnDict = json.dumps(ConnNodeDict)
BaseTermDict = json.dumps(TerminalsDict)

In [ ]:
Tree={}
#ProcessedNodes=[]
TotalNodes=0
StartTime = time.process_time()

# Iterate through all substation transformers
for i6 in range(len(SwitchKeys)):

    FirstNode = 1 
    LastNode = 2
    # Set as rootnode 
    # assuming node1 is high and node2 is low - need to verify will work otherwise
    Tree[SwitchKeys[i6]]=[SwitchDict[SwitchKeys[i6]]['node1'], SwitchDict[SwitchKeys[i6]]['node2']]

    while LastNode != FirstNode:
        NextTerm = ConnNodeDict[Tree[SwitchKeys[i6]][FirstNode]]['list']
        FirstNode = FirstNode + 1
        while NextTerm != 0:
            # Get next node and terminal for current node
            NextNode = TerminalsDict[TermList[NextTerm-1]]['far']
            NextTerm = TerminalsDict[TermList[NextTerm-1]]['next']
            # Add to tree if not there already
            not_in_tree = True

            for i7 in range(len(Tree.keys())):
                if NodeList[NextNode-1] in Tree[SwitchKeys[i7]]:
                    not_in_tree = False
                    break


            if not_in_tree:       
                if 'nomv' in ConnNodeDict[NodeList[NextNode-1]]:
                    if int(ConnNodeDict[NodeList[NextNode-1]]['nomv']) < 34000:
                        Tree[SwitchKeys[i6]].append(NodeList[NextNode-1])
                        LastNode = LastNode + 1
                else:
                    Tree[SwitchKeys[i6]].append(NodeList[NextNode-1])
                    LastNode = LastNode + 1

    NodesInTree=len(Tree[SwitchKeys[i6]])
    print("Processed topology from  ", SwitchDict[SwitchKeys[i6]]['name'], " with id ", SwitchKeys[i6], ' with ', NodesInTree, " buses")
    TotalNodes=TotalNodes+NodesInTree

print("Processed ", len(Tree.keys()), "topology trees containing ", TotalNodes, " buses in ", time.process_time() - StartTime, "seconds")


# ## Build Spanning Tree from DGs



Subs_old=list(Tree.keys())
TotalNodes = 0
StartTime = time.process_time()



In [ ]:
switch = '_1962A006-8FA7-403E-A06F-DCBFE991E771'
for i8 in range(len(Tree[switch])):
    print(ConnNodeDict[Tree[switch][i8]]['name'])